## Keyword-Only Arguments — Practice Set (Advanced, not too much)

Work through each task below. Keyword-only parameters are enforced using the `*` marker (and occasionally `**extras`). Each task has brief tests with `assert` so you can immediately validate behavior.

### Task 1 — `safe_divide`
**Signature:** `safe_divide(a, b, *, on_zero='raise', ndigits=None)`

- Normal division.
- `on_zero`: `'raise' | 'none' | 'inf'` controls division-by-zero behavior.
- `ndigits`: optional rounding; must be keyword-only.

In [1]:
from typing import Any, Dict, Iterable, List, Mapping, Optional, Sequence, Tuple, Callable

def safe_divide(a: float, b: float, *, on_zero: str = "raise", ndigits: Optional[int] = None) -> Optional[float]:
    """Divide a by b with explicit keyword-only behavior for zero-div handling and rounding."""
    if b == 0:
        if on_zero == "raise":
            raise ZeroDivisionError("division by zero")
        elif on_zero == "none":
            return None
        elif on_zero == "inf":
            if a > 0:
                return float("inf")
            if a < 0:
                return -float("inf")
            return 0.0
        else:
            raise ValueError("on_zero must be 'raise', 'none', or 'inf'")
    result = a / b
    return round(result, ndigits) if ndigits is not None else result

# tests
assert safe_divide(10, 4) == 2.5
assert safe_divide(10, 0, on_zero="none") is None
assert safe_divide(10, 0, on_zero="inf") == float("inf")
assert safe_divide(-10, 0, on_zero="inf") == -float("inf")
assert safe_divide(1, 3, ndigits=2) == 0.33

### Task 2 — `windowed_mean`
**Signature:** `windowed_mean(data, *, start=None, end=None)`

- Compute the mean of `data[start:end]`.
- Raise `ValueError` if the window is empty.
- Indices must be passed as keywords.

In [2]:
def windowed_mean(data: Sequence[float], *, start: Optional[int] = None, end: Optional[int] = None) -> float:
    """Mean of a slice of data; start/end are keyword-only and mirror slicing semantics."""
    window = data[slice(start, end)]
    if len(window) == 0:
        raise ValueError("empty window")
    return sum(window) / len(window)

# tests
nums = [10, 20, 30, 40, 50]
assert windowed_mean(nums, start=1, end=4) == (20 + 30 + 40) / 3
try:
    _ = windowed_mean(nums, start=5)
    assert False, "expected ValueError"
except ValueError:
    pass

### Task 3 — `build_url`
**Signature:** `build_url(base, *, path="", query=None, **extras)`

- Join `base` and `path` safely (no double slashes).
- Render query string from `query` mapping; `extras` add params that don't overwrite `query`.

In [3]:
def build_url(base: str, *, path: str = "", query: Optional[Mapping[str, Any]] = None, **extras: Any) -> str:
    """Construct a URL with keyword-only knobs; extras extend query but don't override."""
    b = base.rstrip("/")
    p = path.strip("/")
    url = f"{b}/{p}" if p else b

    params: Dict[str, Any] = {}
    if query:
        params.update(query)
    for k, v in extras.items():
        if k not in params:
            params[k] = v

    if params:
        parts = [f"{k}={v}" for k, v in params.items()]
        return f"{url}?{'&'.join(parts)}"
    return url

# tests
assert build_url("https://api.example.com", path="/v1/users/") == "https://api.example.com/v1/users"
assert build_url("x", path="y", query={"a": 1}, a=2, b=3) in {"x/y?a=1&b=3", "x/y?b=3&a=1"}
assert build_url("x") == "x"

### Task 4 — `log_line`
**Signature:** `log_line(message, *, level='INFO', ts=None, **context)`

- Format like: `[LEVEL] message | ts=... key1=val1 key2=val2`.
- `level` and `ts` are keyword-only; `context` is optional metadata.

In [4]:
def log_line(message: str, *, level: str = "INFO", ts: Optional[str] = None, **context: Any) -> str:
    """Format a log line; do not require context keys."""
    head = f"[{level}] {message}"
    parts = []
    if ts is not None:
        parts.append(f"ts={ts}")
    for k, v in context.items():
        parts.append(f"{k}={v}")
    return f"{head} | " + " ".join(parts) if parts else head

# tests
s = log_line("started", level="DEBUG", ts="2025-11-05T10:00:00Z", user="alice", req=42)
assert s.startswith("[DEBUG] started | ")
assert "ts=2025-11-05T10:00:00Z" in s and "user=alice" in s and "req=42" in s

### Task 5 — `normalize_names`
**Signature:** `normalize_names(*names, *, case='title', sep=', ')`

- Accept any number of positional names and normalize them.
- `case` in `{'lower','upper','title'}`; `sep` is the joiner.

In [5]:
def normalize_names(*names: str, case: str = "title", sep: str = ", ") -> str:
    """Normalize and join names with keyword-only formatting options."""
    def transform(s: str) -> str:
        if case == "lower":
            return s.lower()
        if case == "upper":
            return s.upper()
        if case == "title":
            return s.title()
        raise ValueError("case must be 'lower', 'upper', or 'title'")
    return sep.join(transform(n) for n in names)

# tests
assert normalize_names("joHN", "doe") == "John, Doe"
assert normalize_names("a", "b", "c", case="upper", sep="|") == "A|B|C"

### Task 6 — `coords_to_geojson`
**Signature:** `coords_to_geojson(*, longitude, latitude, **properties)`

- Enforce lon/lat as keyword-only.
- Return a minimal GeoJSON Feature (Point) string; extras go in `properties`.

In [6]:
import json

def coords_to_geojson(*, longitude: float, latitude: float, **properties: Any) -> str:
    """Return a minimal GeoJSON Feature (Point) as a JSON string.
    lon/lat are keyword-only; any extra keyword args go into properties.
    """
    feature = {
        "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [longitude, latitude]},
    }
    if properties:
        feature["properties"] = properties
    return json.dumps(feature, separators=(",", ":"))  # compact JSON

# tests (simple substring checks)
g = coords_to_geojson(longitude=10, latitude=20, name="HQ", active=True)
assert '"coordinates"' in g and '[10,20]' in g
assert '"name":"HQ"' in g and '"active":true' in g


### Task 7 — `export_rows`
**Signature:** `export_rows(rows, *, field_sep=',', line_sep='\n', header=None)`

- CSV-like export; header is optional.
- All knobs are keyword-only.

In [7]:
def export_rows(
    rows: Iterable[Sequence[Any]],
    *,
    field_sep: str = ",",
    line_sep: str = "\n",
    header: Optional[Sequence[str]] = None,
) -> str:
    """CSV-like export with keyword-only formatting and optional header."""
    def fmt_row(r: Sequence[Any]) -> str:
        return field_sep.join(str(x) for x in r)
    chunks: List[str] = []
    if header is not None:
        chunks.append(fmt_row(header))
    for r in rows:
        chunks.append(fmt_row(r))
    return line_sep.join(chunks)

# tests
data = [(1, 2), (3, 4)]
out = export_rows(data, header=("A", "B"), field_sep=";", line_sep="|")
assert out == "A;B|1;2|3;4"

### Task 8 — `polygon_area`
**Signature:** `polygon_area(*, points, signed=False)`

- Compute area via shoelace formula.
- If `signed=True`, preserve sign; otherwise return absolute value.

In [8]:
def polygon_area(*, points: Sequence[Tuple[float, float]], signed: bool = False) -> float:
    """Compute polygon area; points are keyword-only to avoid positional confusion."""
    if len(points) < 3:
        raise ValueError("need at least 3 points")
    s = 0.0
    n = len(points)
    for i in range(n):
        x1, y1 = points[i]
        x2, y2 = points[(i + 1) % n]
        s += x1 * y2 - x2 * y1
    area = 0.5 * s
    return area if signed else abs(area)

# tests (unit square)
square = [(0,0), (1,0), (1,1), (0,1)]
assert polygon_area(points=square) == 1.0
assert polygon_area(points=square, signed=True) in (1.0, -1.0)

### Task 9 — `print_wrapper`
**Signature:** `print_wrapper(*values, *, sep=' ', end='\n', file=None)`

- Forward to built-in `print` while enforcing keyword-only formatting knobs.

In [9]:
def print_wrapper(*values: Any, sep: str = " ", end: str = "\n", file: Any = None) -> None:
    """Thin wrapper around print enforcing keyword-only formatting knobs."""
    print(*values, sep=sep, end=end, file=file)

# test (visual)
# print_wrapper("A", "B", "C", sep="|", end="!\n")  # -> A|B|C!

### Task 10 — `map_apply`
**Signature:** `map_apply(fn, *iterables, *, unpack=False)`

- If `unpack=False`, expect exactly one iterable and call `fn(item)`.
- If `unpack=True`, expect one iterable of tuples and call `fn(*item)`.

In [10]:
def map_apply(fn: Callable[..., Any], *iterables: Iterable[Any], unpack: bool = False) -> List[Any]:
    """Apply fn over items with an explicit keyword-only unpack switch."""
    if not iterables:
        raise ValueError("provide at least one iterable")
    if unpack:
        if len(iterables) != 1:
            raise ValueError("with unpack=True, provide exactly one iterable of tuples")
        return [fn(*item) for item in iterables[0]]
    else:
        if len(iterables) != 1:
            raise ValueError("with unpack=False, provide exactly one iterable")
        return [fn(item) for item in iterables[0]]

# tests
pairs = [(2, 5), (3, 7)]
assert map_apply(lambda x, y: x + y, pairs, unpack=True) == [7, 10]
assert map_apply(lambda s: s.upper(), ["a", "b"]) == ["A", "B"]

### Task 11 — `merge_configs`
**Signature:** `merge_configs(base, *, override=None, **patches)`

- Merge layers with clear precedence: `base < override < patches`.
- Do not mutate inputs; never require specific keys in `**patches`.

In [11]:
def merge_configs(base: Mapping[str, Any], *, override: Optional[Mapping[str, Any]] = None, **patches: Any) -> Dict[str, Any]:
    """Merge configuration layers with clear precedence and keyword-only knobs."""
    result = dict(base)
    if override:
        result.update(override)
    if patches:
        result.update(patches)
    return result

# tests
b = {"a": 1, "b": 2}
ov = {"b": 20}
mc = merge_configs(b, override=ov, c=3)
assert mc == {"a": 1, "b": 20, "c": 3}
assert b == {"a": 1, "b": 2} and ov == {"b": 20}  # unchanged

### Task 12 — `make_counter` (factory)
**Signature:** `make_counter(*, start=0, step=1)`

- Return a function with no parameters that yields the next value on each call.
- Tuning parameters must be keyword-only.

In [12]:
def make_counter(*, start: int = 0, step: int = 1) -> Callable[[], int]:
    """Return a stateful counter; all tuning is keyword-only."""
    current = start
    def inc() -> int:
        nonlocal current
        value = current
        current += step
        return value
    return inc

# tests
c = make_counter(start=10, step=3)
assert [c(), c(), c()] == [10, 13, 16]

### BONUS — Validate keyword-only enforcement
The calls below should raise `TypeError` because keyword-only parameters were passed positionally.

In [13]:
try:
    _ = safe_divide(1, 0, "none")  # missing keyword for on_zero
    assert False, "expected TypeError"
except TypeError:
    pass

try:
    _ = windowed_mean([1,2,3], 0, 2)  # start/end must be named
    assert False, "expected TypeError"
except TypeError:
    pass

try:
    _ = coords_to_geojson(10, 20)  # lon/lat must be named
    assert False, "expected TypeError"
except TypeError:
    pass

print("\u2705 All keyword-only practice tests passed (visual ones skipped).")

✅ All keyword-only practice tests passed (visual ones skipped).
